# Sizing for a Simple OTA with PMOS Active Load {.unnumbered .unlisted}

**Analysis of a differential amplifier based on the provided netlist.**

M1/M2: Differential input pair (NMOS)
M7/M8: PMOS active load (current mirror)
M3/M4: Reference leg of the PMOS current mirror
M11: Tail current source for the input pair
M5/M6: Common-Mode Feedback (CMFB) circuit transistors
M9, M10, M12: Bias current mirror network

In [1]:
# read table data
from pygmid import Lookup as lk
import numpy as np
lv_nmos = lk('sg13_lv_nmos.mat')
lv_pmos = lk('sg13_lv_pmos.mat')
# list of parameters: VGS, VDS, VSB, L, W, NFING, ID, VT, GM, GMB, GDS, CGG, CGB, CGD, CGS, CDD, CSS, STH, SFL

In [2]:
# define the given parameters as taken from the specification table or initial guesses
c_load = 50e-15  # Load capacitance per output (fully differential)
gm_id_m12 = 18    # gm/ID for input differential pair
gm_id_m78 = 15     # gm/ID for PMOS active load
gm_id_m34 = 15     # gm/ID for reference transistors
gm_id_m56 = 10    # gm/ID for CMFB transistors
gm_id_bias = 10   # gm/ID for bias transistors

# Channel lengths
l_12 = 0.5        # Input pair length
l_78 = 0.5        # Active load length
l_34 = 0.5        # Reference transistor length
l_56 = 0.5        # CMFB transistor length
l_bias = 0.5      # Bias transistor length

# Specifications
f_bw = 10e6       # Required unity gain bandwidth
a0_target = 60    # Target DC gain in dB
i_total_limit = 10e-6  # Total current budget
i_bias_in = 100e-6      # Input bias current
output_voltage = 1.3 # Required output voltage level
vin_min = 0.7          # Input voltage minimum 
vin_max = 0.9          # Input voltage maximum 
vdd_min = 1.45         # Supply voltage minimum 
vdd_max = 1.55         # Supply voltage maximum 
vss = 0.0              # Ground
vds_headroom = 0.2     # Minimum VDS for saturation

In [3]:
# Calculate required gm of input pair from bandwidth specification
safety_factor = 3
gm_m12 = f_bw * 2 * np.pi * c_load * safety_factor
print(f'Required gm12 = {gm_m12*1e3:.2f} mS')

# Calculate bias current for input pair
id_m12 = gm_m12 / gm_id_m12
print(f'Input pair bias current = {id_m12*1e6:.2f} µA each')

# Total tail current
i_tail = 2 * id_m12
print(f'Tail current = {i_tail*1e6:.2f} µA')

Required gm12 = 0.01 mS
Input pair bias current = 0.52 µA each
Tail current = 1.05 µA


In [4]:
# Calculate currents for other transistors
# M7/M8: PMOS active load - same current as input pair
id_m78 = id_m12
print(f'Active load current = {id_m78*1e6:.2f} µA each')

# M3/M4: Reference for PMOS current mirror
id_m34 = id_m78  # Same as mirrored current
print(f'Reference current = {id_m34*1e6:.2f} µA each')

# Total supply current (simplified - just the main differential pair branch)
i_total = i_tail + 2*id_m34  # Tail + two reference branches
print(f'Total supply current = {i_total*1e6:.2f} µA')

if i_total < i_total_limit:
    print('[INFO] Power consumption target is met!')
else:
    print('[WARNING] Power consumption target is NOT met!')

Active load current = 0.52 µA each
Reference current = 0.52 µA each
Total supply current = 2.09 µA
[INFO] Power consumption target is met!


In [5]:
# Calculate DC gain
# For a simple OTA with active load: A0 = gm_m12 * (ro_m12 || ro_m78)
# where ro = 1/gds = (gm/gds) / gm

# Look up gm/gds ratios for the input pair and the active load
gm_gds_m12 = lv_nmos.lookup('GM_GDS', GM_ID=gm_id_m12, L=l_12, VDS=vds_headroom)
gm_gds_m78 = lv_pmos.lookup('GM_GDS', GM_ID=gm_id_m78, L=l_78, VDS=vds_headroom)

# Calculate transconductances
gm_m78 = gm_id_m78 * id_m78

# Calculate output resistances
ro_m12 = gm_gds_m12 / gm_m12
ro_m78 = gm_gds_m78 / gm_m78

# Total output resistance (parallel combination of NMOS and PMOS output resistances)
ro_total = 1 / (1/ro_m12 + 1/ro_m78)

# DC gain
a0 = gm_m12 * ro_total
a0_db = 20 * np.log10(a0)

print(f'DC gain = {a0_db:.1f} dB')
print(f'Output resistance = {ro_total*1e-6:.1f} MΩ')

DC gain = 21.5 dB
Output resistance = 1.3 MΩ


In [6]:
# Calculate parasitic capacitances and actual bandwidth
# Look up capacitance ratios
gm_cgs_m12 = lv_nmos.lookup('GM_CGS', GM_ID=gm_id_m12, L=l_12, VDS=vds_headroom)
gm_cdd_m12 = lv_nmos.lookup('GM_CDD', GM_ID=gm_id_m12, L=l_12, VDS=vds_headroom)
gm_cdd_m78 = lv_pmos.lookup('GM_CDD', GM_ID=gm_id_m78, L=l_78, VDS=vds_headroom)

# Calculate parasitic capacitances
c_parasitic_input = gm_m12 / gm_cgs_m12  # Input capacitance
c_parasitic_output = gm_m12 / gm_cdd_m12 + gm_m78 / gm_cdd_m78  # Output parasitic per node

print(f'Input parasitic capacitance = {c_parasitic_input*1e15:.1f} fF')
print(f'Output parasitic capacitance = {c_parasitic_output*1e15:.1f} fF per output')

# Actual unity gain bandwidth including parasitics
c_total = c_load + c_parasitic_output
f_ugb_actual = gm_m12 / (2 * np.pi * c_total)
print(f'Actual unity gain bandwidth = {f_ugb_actual*1e-6:.1f} MHz')

# Phase margin (simplified estimate)
# Dominant pole at output, second pole approximately at gm/C_parasitic
f_p2 = min(gm_m12, gm_m78) / (2 * np.pi * c_parasitic_output)
phase_margin = 90 - np.arctan(f_ugb_actual / f_p2) * 180 / np.pi
print(f'Estimated phase margin = {phase_margin:.1f}°')

Input parasitic capacitance = -0.4 fF
Output parasitic capacitance = 0.8 fF per output
Actual unity gain bandwidth = 29.5 MHz
Estimated phase margin = 88.9°


In [7]:
# Look up VGS values for biasing
vgs_m12 = lv_nmos.look_upVGS(GM_ID=gm_id_m12, L=l_12, VDS=vds_headroom)
vgs_m78 = lv_pmos.look_upVGS(GM_ID=gm_id_m78, L=l_78, VDS=vds_headroom)
vgs_m34 = lv_pmos.look_upVGS(GM_ID=gm_id_m34, L=l_34, VDS=vds_headroom)
vgs_m56 = lv_nmos.look_upVGS(GM_ID=gm_id_m56, L=l_56, VDS=vds_headroom)
vgs_bias = lv_nmos.look_upVGS(GM_ID=gm_id_bias, L=l_bias, VDS=vds_headroom)

print(f'VGS M1/M2 = {float(vgs_m12):.3f} V')
print(f'VGS M7/M8 = {float(vgs_m78):.3f} V')
print(f'VGS M3/M4 = {float(vgs_m34):.3f} V')
print(f'VGS M5/M6 = {float(vgs_m56):.3f} V')
print(f'VGS bias = {float(vgs_bias):.3f} V')

VGS M1/M2 = 0.325 V
VGS M7/M8 = 0.447 V
VGS M3/M4 = 0.447 V
VGS M5/M6 = 0.446 V
VGS bias = 0.446 V


In [8]:
# Calculate transistor widths
# M1/M2: Input differential pair
id_w_m12 = lv_nmos.lookup('ID_W', GM_ID=gm_id_m12, L=l_12, VDS=vds_headroom)
w_m12 = id_m12 / id_w_m12
w_m12_round = max(round(w_m12 * 2) / 2, 0.5)
print(f'M1/M2 W = {w_m12:.2f} µm, rounded = {w_m12_round} µm')

# M7/M8: PMOS Active Load
id_w_m78 = lv_pmos.lookup('ID_W', GM_ID=gm_id_m78, L=l_78, VDS=vds_headroom)
w_m78 = id_m78 / id_w_m78
w_m78_round = max(round(w_m78 * 2) / 2, 0.5)
print(f'M7/M8 W = {w_m78:.2f} µm, rounded = {w_m78_round} µm')

# M3/M4: Reference for current mirror
id_w_m34 = lv_pmos.lookup('ID_W', GM_ID=gm_id_m34, L=l_34, VDS=vds_headroom)
w_m34 = id_m34 / id_w_m34
w_m34_round = max(round(w_m34 * 2) / 2, 0.5)
print(f'M3/M4 W = {w_m34:.2f} µm, rounded = {w_m34_round} µm')

# M5/M6: CMFB transistors
id_m56 = id_m12
id_w_m56 = lv_nmos.lookup('ID_W', GM_ID=gm_id_m56, L=l_56, VDS=vds_headroom)
w_m56 = id_m56 / id_w_m56  # Assuming same current as input for CMFB
w_m56_round = max(round(w_m56 * 2) / 2, 0.5)
print(f'M5/M6 W = {w_m56:.2f} µm, rounded = {w_m56_round} µm')

# Bias transistors
# M11: Tail current source
id_w_bias = lv_nmos.lookup('ID_W', GM_ID=gm_id_bias, L=l_bias, VDS=vds_headroom)
w_m11 = i_tail / id_w_bias
w_m11_round = max(round(w_m11 * 2) / 2, 0.5)
print(f'M11 (tail) W = {w_m11:.2f} µm, rounded = {w_m11_round} µm')

# M9/M10/M12: Other bias transistors
w_m9 = w_m11_round * i_bias_in / i_tail
w_m9_round = max(round(w_m9 * 2) / 2, 0.5)
print(f'M9/M10/M12 W = {w_m9:.2f} µm, rounded = {w_m9_round} µm')

M1/M2 W = 0.29 µm, rounded = 0.5 µm
M7/M8 W = 0.63 µm, rounded = 0.5 µm
M3/M4 W = 0.63 µm, rounded = 0.5 µm
M5/M6 W = 0.06 µm, rounded = 0.5 µm
M11 (tail) W = 0.11 µm, rounded = 0.5 µm
M9/M10/M12 W = 47.75 µm, rounded = 47.5 µm


In [9]:
# Calculate noise performance
# Look up thermal noise spectral density to find gamma
sth_m12_lookup = lv_nmos.lookup('STH_GM', VGS=vgs_m12, L=l_12, VDS=vds_headroom)
sth_m78_lookup = lv_pmos.lookup('STH_GM', VGS=vgs_m78, L=l_78, VDS=vds_headroom)

# Calculate gamma factors (noise coefficients)
# Note: A sth lookup value of 0 indicates it may be out of the model's range.
gamma_m12 = sth_m12_lookup / (4 * 1.38e-23 * 300) if sth_m12_lookup > 0 else 2/3
gamma_m78 = sth_m78_lookup / (4 * 1.38e-23 * 300) if sth_m78_lookup > 0 else 2/3

# Input referred noise voltage spectral density (V^2/Hz)
# Contribution from input pair (M1, M2) and active load (M7, M8)
# S_v_in^2 = (8*k*T/gm_m12^2) * (gamma_n*gm_m12 + gamma_p*gm_m78)
input_noise_psd = (8 * 1.38e-23 * 300 / gm_m12**2) * (gamma_m12 * gm_m12 + gamma_m78 * gm_m78)
input_noise_rms = np.sqrt(input_noise_psd * f_ugb_actual)

print(f'Input referred noise PSD = {input_noise_psd*1e18:.1f} nV²/Hz')
print(f'Input referred noise (RMS) = {input_noise_rms*1e9:.1f} nVrms')

# Output referred noise
output_noise_rms = input_noise_rms * a0
print(f'Output referred noise (RMS) = {output_noise_rms*1e6:.1f} µVrms')

Input referred noise PSD = 6585.0 nV²/Hz
Input referred noise (RMS) = 440798.2 nVrms
Output referred noise (RMS) = 5243.2 µVrms


In [10]:
# Calculate settling time
# Slewing time
t_slew = c_total * output_voltage / i_tail
print(f'Slewing time = {t_slew*1e9:.1f} ns')

# Linear settling time (5 time constants)
t_settle = 5 / (2 * np.pi * f_ugb_actual)
print(f'Linear settling time = {t_settle*1e9:.1f} ns')

# Total settling time
t_total = t_slew + t_settle
print(f'Total settling time = {t_total*1e9:.1f} ns')

Slewing time = 63.1 ns
Linear settling time = 27.0 ns
Total settling time = 90.1 ns


In [11]:
# Voltage headroom analysis
print('\nVoltage Headroom Analysis:')
print('=' * 30)

# Look up threshold voltage for the input NMOS transistor
vt_nmos = lv_nmos.lookup('VT', VGS=vgs_m12, L=l_12, VDS=vds_headroom)

# Input common mode range
# VCM_min is limited by keeping the tail current source (M11) in saturation
vcm_min = vgs_m12 + vds_headroom
# VCM_max is limited by keeping the input pair (M1/M2) in saturation
vcm_max = vdd_min - abs(vgs_m78) + vt_nmos
print(f'Input CM range: {vcm_min:.3f} V to {vcm_max:.3f} V')

# Output swing analysis
# Vout_min is limited by keeping M1/M2 in saturation
vout_min = vcm_min - vt_nmos
# Vout_max is limited by keeping the PMOS load (M7/M8) in saturation
vout_max = vdd_min - vds_headroom
output_swing_actual = vout_max - vout_min
print(f'Output swing: {output_swing_actual:.3f} V ({vout_min:.3f} V to {vout_max:.3f} V)')

# Check if specifications are met
if vcm_min <= vin_min and vin_max <= vcm_max:
    print('[INFO] Input voltage range requirement is met!')
else:
    print('[WARNING] Input voltage range requirement is NOT met!')

if vout_max >= output_voltage:
    print('[INFO] Output voltage requirement is met!')
else:
    print('[WARNING] Output voltage requirement is NOT met!')


Voltage Headroom Analysis:
Input CM range: 0.525 V to 1.275 V
Output swing: 0.997 V (0.253 V to 1.250 V)
[INFO] Input voltage range requirement is met!
[WARNING] Output voltage requirement is NOT met!


In [12]:
# Final design summary
print('\n' + '=' * 50)
print('SIMPLE OTA WITH PMOS ACTIVE LOAD DESIGN')
print('=' * 50)

print('\nTransistor Dimensions:')
print('-------------------------')
print('# NOTE: These dimensions are from the original notebook and have not been recalculated.')
print(f'M1/M2 (input pair):     W = {w_m12_round} µm, L = {l_12} µm')
print(f'M7/M8 (active load):    W = {w_m78_round} µm, L = {l_78} µm')
print(f'M3/M4 (reference):      W = {w_m34_round} µm, L = {l_34} µm')
print(f'M5/M6 (CMFB):           W = {w_m56_round} µm, L = {l_56} µm')
print(f'M11 (tail current):     W = {w_m11_round} µm, L = {l_bias} µm')
print(f'M9/M10/M12 (bias):      W = {w_m9_round} µm, L = {l_bias} µm')

print('\nPerformance Summary:')
print('-------------------------')
print(f'Supply current:         {i_total*1e6:.1f} µA')
print(f'DC gain:                {a0_db:.1f} dB')
print(f'Unity gain bandwidth:   {f_ugb_actual*1e-6:.1f} MHz')
print(f'Phase margin:           {phase_margin:.1f}°')
print(f'Input noise (RMS):      {input_noise_rms*1e9:.1f} nVrms')
print(f'Output swing:           {output_swing_actual:.3f} V')
print(f'Settling time:          {t_total*1e9:.1f} ns')
print(f'Input CM range:         {vcm_min:.3f} V to {vcm_max:.3f} V')

print('\nBias Voltages:')
print('-------------------------')
print(f'VGS M1/M2:              {float(vgs_m12):.3f} V')
print(f'VGS M7/M8:              {float(vgs_m78):.3f} V')
print(f'VGS M3/M4:              {float(vgs_m34):.3f} V')
print(f'VGS M5/M6:              {float(vgs_m56):.3f} V')
print(f'VGS bias:               {float(vgs_bias):.3f} V')

print('\n' + '=' * 50)

# Assuming a typical input CM voltage for checking saturation
vcm_typical = (vin_min + vin_max) / 2

print('\nVDS Saturation Check:')
print('-------------------------')
# Vds for the tail transistor M11
vds_m11 = vcm_typical - vgs_m12
print(f'VDS M11 = {vds_m11:.3f} V (must be > {vds_headroom} V)')

# Vds for the active load M7/M8
# Assuming output is at the specified DC level
vds_m78 = output_voltage - vdd_min
print(f'VDS M7/M8 = {vds_m78:.3f} V (must be < -{vds_headroom} V)')


SIMPLE OTA WITH PMOS ACTIVE LOAD DESIGN

Transistor Dimensions:
-------------------------
# NOTE: These dimensions are from the original notebook and have not been recalculated.
M1/M2 (input pair):     W = 0.5 µm, L = 0.5 µm
M7/M8 (active load):    W = 0.5 µm, L = 0.5 µm
M3/M4 (reference):      W = 0.5 µm, L = 0.5 µm
M5/M6 (CMFB):           W = 0.5 µm, L = 0.5 µm
M11 (tail current):     W = 0.5 µm, L = 0.5 µm
M9/M10/M12 (bias):      W = 47.5 µm, L = 0.5 µm

Performance Summary:
-------------------------
Supply current:         2.1 µA
DC gain:                21.5 dB
Unity gain bandwidth:   29.5 MHz
Phase margin:           88.9°
Input noise (RMS):      440798.2 nVrms
Output swing:           0.997 V
Settling time:          90.1 ns
Input CM range:         0.525 V to 1.275 V

Bias Voltages:
-------------------------
VGS M1/M2:              0.325 V
VGS M7/M8:              0.447 V
VGS M3/M4:              0.447 V
VGS M5/M6:              0.446 V
VGS bias:               0.446 V


VDS Saturation